In [ ]:
!nvidia-smi

Mon Sep 27 09:24:19 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    71W / 149W |   4523MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
path = '/content/gdrive/MyDrive/emlo_session2_model_30epochs.pt'

In [ ]:
import torch
import torch.nn as nn 
from torchvision import models 
from torchsummary import summary
from torchvision import transforms, datasets
import numpy as np
from tqdm import tqdm
from functools import partial

tqdm = partial(tqdm, position=0, leave=True)

In [ ]:
# model = models.resnet18(pretrained=False, num_classes=10)
# model.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1, bias=False)
# model.maxpool = nn.Identity()
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = models.resnet18(pretrained=False, num_classes=10)
        # for param in self.model.parameters():
        #     param.requires_grad = False
        self.model.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1, bias=False)
        self.model.maxpool = nn.Identity()
        
        # self.model.fc = nn.Linear(self.model.fc.in_features, 10)

    
    def forward(self, x):
        return self.model(x)

In [ ]:
train_transform = transforms.Compose(transforms= [
    transforms.RandomCrop(32, padding=4),
    transforms.Resize((32,32)),

    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.48216, 0.44653], std=[0.2023, 0.1994, 0.2010])
    
])
valid_transforms = transforms.Compose(transforms=[
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.48216, 0.44653], std=[0.2023, 0.1994, 0.2010])
   
])

In [ ]:
batch_size=128

trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
validset = datasets.CIFAR10(root='./data', train=False, download=True, transform=valid_transforms)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
valid_loader = torch.utils.data.DataLoader(validset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
model = Model()
epochs=30
optimizer = torch.optim.SGD(model.parameters(), lr=0.05, momentum=0.9)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,epochs)

criterion = nn.CrossEntropyLoss()


In [ ]:
DEVICE='cuda'
model.to(DEVICE)
def train(model, train_loader, optimizer, loss_fn, scheduler=None,device=DEVICE):
    model.train()
    correct = 0
    train_loss = 0

    for _, data in tqdm(enumerate(train_loader), total=len(train_loader)):
        
        data, target = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()

        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        train_loss += loss.detach().item()
        loss.backward()
        optimizer.step()
        if scheduler is not None:
          scheduler.step()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()


    # train_loss.append(train_loss * 1.0 / len(train_loader.dataset))
    # train_acc.append(correct * 100.0 / len(train_loader.dataset))

    print(
        f"Avg Train loss = {train_loss * 1.0 / len(train_loader.dataset)}, Train Accuracy : {100.0 * correct / len(train_loader.dataset)}"
    )

def evaluate(model, valid_loader, loss_fn, device=DEVICE):
    model.eval()
    correct = 0
    valid_loss = 0
    
    with torch.no_grad():
        for _, data in tqdm(enumerate(valid_loader), total=len(valid_loader)):
            data, target = data[0].to(device), data[1].to(device)
            output = model(data)
            valid_loss += loss_fn(output, target).item()
            pred = output.argmax(dim=1, keepdims=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    
    valid_loss /= len(valid_loader.dataset) * 1.0
    # valid_loss.append(valid_loss)
    # valid_acc.append(100.0 * correct / len(valid_loader.dataset))
    
    print(
        f"Avg Valid loss = {valid_loss}, Valid Accuracy : {100.0 * correct / len(valid_loader.dataset)}"
    )

In [ ]:
for i in range(epochs):
    print(f'EPOCH {i+1}')
    train(model, train_loader, optimizer, loss_fn=criterion, scheduler=scheduler)
    evaluate(model,valid_loader, criterion )
    # scheduler.step()

EPOCH 1


100%|██████████| 391/391 [03:13<00:00,  2.02it/s]

Avg Train loss = 0.012973487017154693, Train Accuracy : 39.512



100%|██████████| 79/79 [00:06<00:00, 12.07it/s]

Avg Valid loss = 0.010098959028720855, Valid Accuracy : 53.66
EPOCH 2


...

Avg Valid loss = 0.0036653532922267915, Valid Accuracy : 87.62
EPOCH 30



100%|██████████| 391/391 [03:13<00:00,  2.02it/s]

Avg Train loss = 0.001136110919341445, Train Accuracy : 94.928



100%|██████████| 79/79 [00:06<00:00, 11.99it/s]

Avg Valid loss = 0.003147231647372246, Valid Accuracy : 89.44


In [ ]:
 torch.save(model.state_dict(),path)

In [ ]:
print('done!')

done!


In [ ]:
m2 = Model()
m2.load_state_dict(torch.load(path))

<All keys matched successfully>